# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("weather_data.csv")
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint Paul Harbor,57.79,-152.41,53.60,66,1,4.23,US,1599972485
1,1,Zhigalovo,54.81,105.16,49.77,90,100,2.28,RU,1599972487
2,2,Qaanaaq,77.48,-69.36,23.99,85,97,9.10,GL,1599972488
3,3,Kruisfontein,-34.00,24.73,62.01,80,98,5.01,ZA,1599972489
4,4,Ixtapa,20.70,-105.20,80.01,88,75,4.70,MX,1599972490


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
fit_weather = weather_data.loc[(weather_data['Max Temp']>80)
                               & (weather_data['Max Temp']<100) 
                               & (weather_data['Humidity']<80)
                               & (weather_data['Cloudiness']<1)
                               & (weather_data['Wind Speed']<10),:]


fit_weather.reset_index(inplace=True)
del fit_weather['Unnamed: 0']
del fit_weather['index']
fit_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ridgecrest,35.62,-117.67,80.01,16,0,2.55,US,1599972451
1,Makkah al Mukarramah,21.43,39.83,97.30,19,0,9.78,SA,1599972613
2,Palmas,-10.21,-48.36,80.60,30,0,9.17,BR,1599972698
3,La Providencia,28.72,-111.58,84.00,34,0,1.01,MX,1599972904
4,Shache,38.42,77.24,81.18,17,0,7.70,CN,1599972855
5,Charthāwal,29.55,77.58,89.73,55,0,4.21,IN,1599973147
6,Mulegé,26.88,-111.98,82.15,42,0,1.28,MX,1599973244
7,Mount Isa,-20.73,139.50,87.80,18,0,9.17,AU,1599973348
8,Kidal,18.44,1.41,86.45,21,0,0.83,ML,1599973385
9,Cocorit,27.58,-109.97,82.09,52,0,3.29,MX,1599973444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
fit_weather["Hotel Name"] = ""
hotel_df = fit_weather
hotel_df.head()


C:\Users\ryana\anaconda3\envs\bcs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ridgecrest,35.62,-117.67,80.01,16,0,2.55,US,1599972451,
1,Makkah al Mukarramah,21.43,39.83,97.30,19,0,9.78,SA,1599972613,
2,Palmas,-10.21,-48.36,80.60,30,0,9.17,BR,1599972698,
3,La Providencia,28.72,-111.58,84.00,34,0,1.01,MX,1599972904,
4,Shache,38.42,77.24,81.18,17,0,7.70,CN,1599972855,


In [20]:
hotel_list = []
for city in range(len(hotel_df["City"])):
    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]
    
    city_coord = f"{lat},{lng}"
    
    params = {
        "location": city_coord,
        "types":"lodging",
        "radius": 5000,
        "key":g_key
        
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_request = requests.get(base_url, params = params).json()
    try:
        hotel_list.append(hotel_request['results'][0]['name'])
    except:
        hotel_list.append("No Hotels Found")
hotel_df["Hotel Name"] = hotel_list
hotel_df

C:\Users\ryana\anaconda3\envs\bcs\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ridgecrest,35.62,-117.67,80.01,16,0,2.55,US,1599972451,SpringHill Suites by Marriott Ridgecrest
1,Makkah al Mukarramah,21.43,39.83,97.30,19,0,9.78,SA,1599972613,Hotel Fairmont Makkah Clock Royal Tower
2,Palmas,-10.21,-48.36,80.60,30,0,9.17,BR,1599972698,Hotel Lago da Palma
3,La Providencia,28.72,-111.58,84.00,34,0,1.01,MX,1599972904,No Hotels Found
4,Shache,38.42,77.24,81.18,17,0,7.70,CN,1599972855,Delong Hotel
5,Charthāwal,29.55,77.58,89.73,55,0,4.21,IN,1599973147,Golden Tant House
6,Mulegé,26.88,-111.98,82.15,42,0,1.28,MX,1599973244,Casa Granada
7,Mount Isa,-20.73,139.50,87.80,18,0,9.17,AU,1599973348,ibis Styles Mt Isa Verona
8,Kidal,18.44,1.41,86.45,21,0,0.83,ML,1599973385,No Hotels Found
9,Cocorit,27.58,-109.97,82.09,52,0,3.29,MX,1599973444,motel kaba


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(
    locations, info_box_content = [f"Nearest Hotel: {hotel}" for hotel in hotel_list])
fig.add_layer(markers)
                        

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))